#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys
import time

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.utils import *
from src.annotator.pos_tagger import *

#### **2. 读取语料**

读取 TSV 格式的平行语料库

In [2]:
# 语料库文件所在目录
data_dir = '../data/raw'

# 读取数据
# limit 参数控制平行句对数量
data = load_data(data_dir, limit=10)
print(f"成功读取 {len(data)} 条平行句对")

# 预览数据
# 第一列：汉语原文，选自《鹿鼎记》
# 第二列：英语译文，选自《The Deer and The Cauldron》（闵福德译）
print("数据前 5 行如下：")
data.head()

成功读取 10 条平行句对
数据前 5 行如下：


,source,target
1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,Along a coastal road somewhere south of the Ya...
2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,In each of the first three carts a single male...
3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,"The four rear carts were occupied by women, th..."
4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,The little girl was crying in a continuous wai...
5,囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！,"One of the soldiers marching alongside, irrita..."


#### **3. 大模型中文词性标注：PKU 赋码集**

In [3]:
# === 加载模型：LLM PKU 赋码集 ===

# 语种：汉语
lang = 'chinese'

# 赋码集：PKU
# https://hanlp.hankcs.com/docs/annotations/pos/pku.html
tagset = 'pku'

# 标注模式：LLM API
mode = 'llm'

# 大模型：deepseek-v3.2 | glm-4.7 | qwen3-max
llm_model = 'deepseek-v3.2'

# 登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/config 文件中设置 LLM_API_KEY=sk-********
t0 = time.time()
llm_zh_pku_tagger = POSTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
    llm_model=llm_model,
)
t1 = time.time()
print('LLM PKU 中文词性标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

LLM PKU 中文词性标注模型加载完毕！
加载耗时：2.21s


In [4]:
# === 标注原文：LLM PKU 赋码集 ===

# 注意：
# 为节省 API 调用成本
# 大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，再次运行程序，时间显示为 0.0 秒
llm_zh_pku_annos = annotate_data(
    data['source'],
    llm_zh_pku_tagger,
)

# 打印标注结果
print("\n=== LLM PKU 中文词性标注结果预览 ===")
display_anno(llm_zh_pku_annos[0])

POS Tagging: 100%|█████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]


=== LLM PKU 中文词性标注结果预览 ===

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
[('江南', 'ns'), ('近', 'a'), ('海滨', 'n'), ('的', 'u'), ('一', 'm'), ('条', 'q'), ('大路', 'n'), ('上', 'f'), ('，', 'w'), ('一', 'm'), ('队', 'q'), ('清兵', 'n'), ('手', 'n'), ('执', 'v'), ('刀枪', 'n'), ('，', 'w'), ('押', 'v'), ('着', 'u'), ('七', 'm'), ('辆', 'q'), ('囚车', 'n'), ('，', 'w'), ('冲风', 'v'), ('冒寒', 'v'), ('，', 'w'), ('向', 'p'), ('北', 'f'), ('而', 'c'), ('行', 'v'), ('。', 'w')]


#### **4. 大模型中文词性标注：CTB 赋码集**

In [5]:
# === 加载模型：LLM CTB 赋码集 ===

# 语种：汉语
lang = 'chinese'

# 赋码集：CTB
# https://hanlp.hankcs.com/docs/annotations/pos/ctb.html
tagset = 'ctb'

# 标注模式：LLM API
mode = 'llm'

# 大模型：deepseek-v3.2 | glm-4.7 | qwen3-max
llm_model = 'deepseek-v3.2'

# 登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/config 文件中设置 LLM_API_KEY=sk-********
t0 = time.time()
llm_zh_ctb_tagger = POSTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
    llm_model=llm_model,
)
t1 = time.time()
print('LLM CTB 中文词性标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

LLM CTB 中文词性标注模型加载完毕！
加载耗时：1.01s


In [6]:
# === 标注原文：LLM CTB 赋码集 ===

# 注意：
# 为节省 API 调用成本
# 大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，再次运行程序，时间显示为 0.0 秒
llm_zh_ctb_annos = annotate_data(
    data['source'],
    llm_zh_ctb_tagger,
)

# 打印标注结果
print("\n=== LLM CTB 中文词性标注结果预览 ===")
display_anno(llm_zh_ctb_annos[0])

POS Tagging: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10060.70it/s]


=== LLM CTB 中文词性标注结果预览 ===

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
[('江南', 'NR'), ('近', 'P'), ('海滨', 'NN'), ('的', 'DEG'), ('一', 'CD'), ('条', 'M'), ('大路', 'NN'), ('上', 'LC'), ('，', 'PU'), ('一', 'CD'), ('队', 'M'), ('清兵', 'NN'), ('手', 'NN'), ('执', 'VV'), ('刀枪', 'NN'), ('，', 'PU'), ('押', 'VV'), ('着', 'AS'), ('七', 'CD'), ('辆', 'M'), ('囚车', 'NN'), ('，', 'PU'), ('冲风', 'VV'), ('冒寒', 'VV'), ('，', 'PU'), ('向', 'P'), ('北', 'NN'), ('而', 'AD'), ('行', 'VV'), ('。', 'PU')]


#### **5. 中文词性标注模型对比：LLM & HanLP**

In [7]:
# === 加载模型：HanLP PKU 赋码集 ===

# 语种：汉语
lang = 'chinese'

# 赋码集：PKU
# https://hanlp.hankcs.com/docs/annotations/pos/pku.html
tagset = 'pku'

# 标注模式：HanlP 本地模型
mode = 'local'

# HanLP PKU 模型：PKU_POS_ELECTRA_SMALL
# https://hanlp.hankcs.com/docs/api/hanlp/pretrained/pos.html
# 首次加载需从网络将模型下载至本地
t0 = time.time()
hanlp_zh_pku_tagger = POSTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
)
t1 = time.time()
print('HanLP PKU 中文词性标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

HanLP PKU 中文词性标注模型加载完毕！
加载耗时：8.33s


In [8]:
# === 标注原文：HanLP PKU 赋码集 ===

hanlp_zh_pku_annos = annotate_data(
    data['source'],
    hanlp_zh_pku_tagger,
)

# 打印标注结果
print("\n=== HanLP PKU 中文词性标注结果预览 ===")
display_anno(hanlp_zh_pku_annos[0])

POS Tagging: 100%|█████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 17.99it/s]


=== HanLP PKU 中文词性标注结果预览 ===

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
[('江南', 'ns'), ('近', 'a'), ('海滨', 's'), ('的', 'u'), ('一', 'm'), ('条', 'q'), ('大', 'a'), ('路上', 's'), ('，', 'w'), ('一', 'm'), ('队', 'q'), ('清兵', 'n'), ('手', 'n'), ('执', 'v'), ('刀枪', 'n'), ('，', 'w'), ('押', 'v'), ('着', 'u'), ('七', 'm'), ('辆', 'q'), ('囚车', 'n'), ('，', 'w'), ('冲', 'v'), ('风', 'n'), ('冒寒', 'v'), ('，', 'w'), ('向', 'p'), ('北', 'f'), ('而', 'c'), ('行', 'v'), ('。', 'w')]


In [9]:
# === 模型对比：LLM PKU & HanLP PKU ===

print('中文词性标注模型对比（PKU 赋码集）：LLM & HanLP')
compare_annos(
    llm_zh_pku_annos,
    hanlp_zh_pku_annos,
    annos_1_name="LLM",
    annos_2_name="HanLP",
)

中文词性标注模型对比（PKU 赋码集）：LLM & HanLP

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
LLM: {('大路', 'n'), ('冲风', 'v'), ('海滨', 'n'), ('上', 'f')}
HanLP: {('风', 'n'), ('冲', 'v'), ('路上', 's'), ('海滨', 's'), ('大', 'a')}

[ID]: 00002
前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。
--------------------------------------------------------------------------------
LLM: {('两个', 'm'), ('一个', 'm'), ('三个', 'm'), ('打扮', 'v')}
HanLP: {('一', 'm'), ('两', 'm'), ('打扮', 'n'), ('个', 'q')}

[ID]: 00003
后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。
--------------------------------------------------------------------------------
LLM: {('最后', 't')}
HanLP: {('最后', 'f')}

[ID]: 00004
女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。
--------------------------------------------------------------------------------
LLM: {('相呵', 'v'), ('不休', 'v'), ('温言', 'n')}
HanLP: {('不休', 'l'), ('温言相呵', 'i')}

[ID]: 00005
囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！
------------------------------------------------

In [10]:
# === 加载模型：HanLP CTB 赋码集 ===

# 语种：汉语
lang = 'chinese'

# 赋码集：CTB
# https://hanlp.hankcs.com/docs/annotations/pos/ctb.html
tagset = 'ctb'

# 标注模式：HanlP 本地模型
mode = 'local'

# HanLP CTB 模型：CTB9_POS_ELECTRA_SMALL
# https://hanlp.hankcs.com/docs/api/hanlp/pretrained/pos.html
# 首次加载需从网络将模型下载至本地
t0 = time.time()
hanlp_zh_ctb_tagger = POSTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
)
t1 = time.time()
print('HanLP CTB 中文词性标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

HanLP CTB 中文词性标注模型加载完毕！
加载耗时：0.59s


In [11]:
# === 标注原文：HanLP CTB 赋码集 ===

hanlp_zh_ctb_annos = annotate_data(
    data['source'],
    hanlp_zh_ctb_tagger,
)

# 打印标注结果
print("\n=== HanLP CTB 中文词性标注结果预览 ===")
display_anno(hanlp_zh_ctb_annos[0])

POS Tagging: 100%|█████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 32.41it/s]


=== HanLP CTB 中文词性标注结果预览 ===

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
[('江南', 'NR'), ('近', 'P'), ('海滨', 'NN'), ('的', 'DEG'), ('一', 'CD'), ('条', 'M'), ('大', 'JJ'), ('路上', 'NN'), ('，', 'PU'), ('一', 'CD'), ('队', 'M'), ('清兵', 'NN'), ('手', 'NN'), ('执', 'VV'), ('刀枪', 'NN'), ('，', 'PU'), ('押', 'VV'), ('着', 'AS'), ('七', 'CD'), ('辆', 'M'), ('囚车', 'NN'), ('，', 'PU'), ('冲', 'VV'), ('风', 'NN'), ('冒寒', 'VV'), ('，', 'PU'), ('向', 'P'), ('北', 'NN'), ('而', 'MSP'), ('行', 'VV'), ('。', 'PU')]


In [12]:
# === 模型对比：LLM CTB & HanLP CTB ===

print('中文词性标注模型对比（CTB 赋码集）：LLM & HanLP')
compare_annos(
    llm_zh_ctb_annos,
    hanlp_zh_ctb_annos,
    annos_1_name="LLM",
    annos_2_name="HanLP",
)

中文词性标注模型对比（CTB 赋码集）：LLM & HanLP

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
LLM: {('而', 'AD'), ('大路', 'NN'), ('上', 'LC'), ('冲风', 'VV')}
HanLP: {('冲', 'VV'), ('而', 'MSP'), ('路上', 'NN'), ('风', 'NN'), ('大', 'JJ')}

[ID]: 00002
前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。
--------------------------------------------------------------------------------
LLM: {('作', 'VV'), ('打扮', 'NN'), ('人', 'NN'), ('中年', 'NN'), ('前面', 'LC')}
HanLP: {('中年人', 'NN'), ('前面', 'NN'), ('打扮', 'VV'), ('作', 'P')}

[ID]: 00003
后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。
--------------------------------------------------------------------------------
LLM: {('最后', 'JJ'), ('抱着', 'VV'), ('后面', 'LC'), ('的', 'SP')}
HanLP: {('抱', 'VV'), ('最后', 'AD'), ('后面', 'NN'), ('的', 'DEC'), ('着', 'AS')}

[ID]: 00004
女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。
--------------------------------------------------------------------------------
LLM: {('不休', 'VA'), ('呵', 'VV'), ('相', 'AD'),